## Tutorial 2: Using a trained AtacWorks model to denoise ATAC-seq data and call peaks. 

## Introduction

In this tutorial we use a pre-trained AtacWorks model to denoise and call peaks from low-coverage aggregate single-cell ATAC-seq data. We use the dsc-ATAC-seq dataset presented in reference (1), section (when text is ready, add a reference to page number, section, table). This dataset consists of single-cell ATAC-seq data from several types of human blood cells.

We selected 2400 NK cells from this dataset - this is our ‘clean’, high-coverage dataset. We then randomly sampled 50 of these 2400 NK cells. Here's what the ATAC-seq signal from 50 cells and 2400 cells looks like, for a region on chromosome 10:

![subsampled_NK_cells](../docs/tutorials/NK.2400.50.png)

Compared to the 'clean' signal from 2400 cells, the aggregated ATAC-seq profile of these 50 cells is noisy. Because the signal is noisy, peak calls calculated by MACS2 on this data (shown as red bars below the signal tracks) are also inaccurate. The AUPRC of peak calling by MACS2 on the noisy data is only 0.20.

As reported in our paper, we trained an AtacWorks model to learn a mapping from 50-cell signal to 2400-cell signals and peak calls. In other words, given a noisy ATAC-seq signal from 50 cells, this model learned what the signal would look like - and where the peaks would be called - if we had sequenced 2400 cells. This model was trained on data from Monocytes and B cells, so it has not encountered data from NK cells.

If you want to train your own AtacWorks model instead of using the model reported in the paper, refer to [Tutorial 1](tutorial1.ipynb).


**NOTE: You  may notice an exclamation mark (!) before most of the commands in this tutorial. That's because most of them are bash commands and to execute bash commands through notebook, they have to be preceded by an exclamation. These commands be directly copy pasted into a terminal (without the !) and executed. We created this notebook to make it very simple for our users to run the tutorials without having to worry about copy pasting.**

## Step 1: Create folder and set AtacWorks path

In [1]:
%env atacworks=/ntadimeti/AtacWorks

env: atacworks=/ntadimeti/AtacWorks


Create a folder for this experiment.  os.chdir() below allows us to enter into the new directory.

In [2]:
!mkdir tutorial2
import os
os.chdir("tutorial2")

## Step 2: Download model

Download a pre-trained deep learning model (model.pth.tar) trained with dsc-ATAC-seq data from Monocytes and B cells. This model was reported and used in the AtacWorks paper (1).

In [3]:
!mkdir models
!wget -P models https://atacworks-paper.s3.us-east-2.amazonaws.com/dsc_atac_blood_cell_denoising_experiments/50_cells/models/model.pth.tar

--2020-06-10 16:30:24--  https://atacworks-paper.s3.us-east-2.amazonaws.com/dsc_atac_blood_cell_denoising_experiments/50_cells/models/model.pth.tar
Resolving atacworks-paper.s3.us-east-2.amazonaws.com (atacworks-paper.s3.us-east-2.amazonaws.com)... 52.219.84.112
Connecting to atacworks-paper.s3.us-east-2.amazonaws.com (atacworks-paper.s3.us-east-2.amazonaws.com)|52.219.84.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 892750 (872K) [application/x-tar]
Saving to: 'models/model.pth.tar'

model.pth.tar       100%[===================>] 871.83K  3.11MB/s    in 0.3s    

2020-06-10 16:30:25 (3.11 MB/s) - 'models/model.pth.tar' saved [892750/892750]



## Step 3: Download config files

We also need to download the 'configs' directory containing config files for this experiment. The config files describe the structure of the deep learning model and the parameters used to run inference.

In [4]:
!mkdir configs
!wget -P configs https://atacworks-paper.s3.us-east-2.amazonaws.com/dsc_atac_blood_cell_denoising_experiments/50_cells/configs/infer_config.yaml
!wget -P configs https://atacworks-paper.s3.us-east-2.amazonaws.com/dsc_atac_blood_cell_denoising_experiments/50_cells/configs/model_structure.yaml

--2020-06-10 16:30:26--  https://atacworks-paper.s3.us-east-2.amazonaws.com/dsc_atac_blood_cell_denoising_experiments/50_cells/configs/infer_config.yaml
Resolving atacworks-paper.s3.us-east-2.amazonaws.com (atacworks-paper.s3.us-east-2.amazonaws.com)... 52.219.104.136
Connecting to atacworks-paper.s3.us-east-2.amazonaws.com (atacworks-paper.s3.us-east-2.amazonaws.com)|52.219.104.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 644 []
Saving to: 'configs/infer_config.yaml'

infer_config.yaml   100%[===================>]     644  --.-KB/s    in 0s      

2020-06-10 16:30:26 (10.9 MB/s) - 'configs/infer_config.yaml' saved [644/644]

--2020-06-10 16:30:26--  https://atacworks-paper.s3.us-east-2.amazonaws.com/dsc_atac_blood_cell_denoising_experiments/50_cells/configs/model_structure.yaml
Resolving atacworks-paper.s3.us-east-2.amazonaws.com (atacworks-paper.s3.us-east-2.amazonaws.com)... 52.219.97.2
Connecting to atacworks-paper.s3.us-east-2.amazonaws.com (atacwo

## Step 4: Download the test dsc-ATAC-seq signal from 50 NK cells (~1M reads), in bigWig format

In [5]:
!wget https://atacworks-paper.s3.us-east-2.amazonaws.com/dsc_atac_blood_cell_denoising_experiments/50_cells/test_data/noisy_data/dsc.1.NK.50.cutsites.smoothed.200.bw

--2020-06-10 16:30:27--  https://atacworks-paper.s3.us-east-2.amazonaws.com/dsc_atac_blood_cell_denoising_experiments/50_cells/test_data/noisy_data/dsc.1.NK.50.cutsites.smoothed.200.bw
Resolving atacworks-paper.s3.us-east-2.amazonaws.com (atacworks-paper.s3.us-east-2.amazonaws.com)... 52.219.97.2
Connecting to atacworks-paper.s3.us-east-2.amazonaws.com (atacworks-paper.s3.us-east-2.amazonaws.com)|52.219.97.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8906319 (8.5M) [binary/octet-stream]
Saving to: 'dsc.1.NK.50.cutsites.smoothed.200.bw'

dsc.1.NK.50.cutsite 100%[===================>]   8.49M  6.52MB/s    in 1.3s    

2020-06-10 16:30:29 (6.52 MB/s) - 'dsc.1.NK.50.cutsites.smoothed.200.bw' saved [8906319/8906319]



## Step 5: Create genomic intervals to define regions for testing

The model we downloaded takes the input ATAC-seq signal in non-overlapping genomic intervals spanning 50,000 bp. To define the genomic regions for the model to read, we take the chromosomes on which we want to apply the model and split their lengths into 50,000-bp intervals, which we save in BED format. 
In this example, we will apply the model to chromosomes 1-22. The reference genome we use is hg19. We use the prepared chromosome sizes file `hg19.auto.sizes`, which contains the sizes of chromosomes 1-22 in hg19.

In [6]:
!mkdir intervals
!python $atacworks/scripts/get_intervals.py \
    --sizes $atacworks/data/reference/hg19.auto.sizes \
    --intervalsize 50000 \
    --out_dir intervals \
    --prefix hg19.50000 \
    --wg

INFO:2020-06-10 16:30:30,413:AtacWorks-intervals] Generating intervals tiling across all chromosomes             in sizes file: /ntadimeti/AtacWorks/data/reference/hg19.auto.sizes
INFO:2020-06-10 16:30:31,152:AtacWorks-intervals] Done!


This produces a BED file (`intervals/hg19.50000.genome_intervals.bed`).

For more information type `python $atacworks/scripts/get_intervals.py --help`

## Step 6: Read test data over the selected intervals, and save in .h5 format

We supply to `bw2h5.py` the bigWig file containing the noisy ATAC-seq signal, and the BED file containing the intervals on which to apply the model. This script reads the ATAC-seq signal within each supplied interval and saves it to a .h5 file.

In [7]:
!python $atacworks/scripts/bw2h5.py \
           --noisybw dsc.1.NK.50.cutsites.smoothed.200.bw \
           --intervals intervals/hg19.50000.genome_intervals.bed \
           --out_dir ./ \
           --prefix NK.50_cells \
           --pad 5000 \
           --nolabel

INFO:2020-06-10 16:30:32,470:AtacWorks-bw2h5] Reading intervals
INFO:2020-06-10 16:30:32,494:AtacWorks-bw2h5] Read 57611 intervals
INFO:2020-06-10 16:30:32,494:AtacWorks-bw2h5] Writing data in 58 batches.
INFO:2020-06-10 16:30:32,494:AtacWorks-bw2h5] Extracting data for each batch and writing to h5 file
INFO:2020-06-10 16:30:32,494:AtacWorks-bw2h5] batch 0 of 58
INFO:2020-06-10 16:31:40,837:AtacWorks-bw2h5] batch 10 of 58
INFO:2020-06-10 16:32:45,415:AtacWorks-bw2h5] batch 20 of 58
INFO:2020-06-10 16:33:50,764:AtacWorks-bw2h5] batch 30 of 58
INFO:2020-06-10 16:34:55,029:AtacWorks-bw2h5] batch 40 of 58
INFO:2020-06-10 16:36:04,497:AtacWorks-bw2h5] batch 50 of 58
INFO:2020-06-10 16:36:54,095:AtacWorks-bw2h5] Done! Saved to ./NK.50_cells.h5


This creates a file `NK.50_cells.h5`, which contains the noisy ATAC-seq signal to be fed to the pre-trained model.

For more information type `python $atacworks/scripts/bw2h5.py --help`

## Step 7: Inference on selected intervals, producing denoised track and binary peak calls

In [8]:
!atacworks denoise \
    --input_files NK.50_cells.h5 \
    --sizes_file $atacworks/data/reference/hg19.auto.sizes \
    --config configs/infer_config.yaml \
    --config_mparams configs/model_structure.yaml

INFO:2020-06-10 16:36:56,424:AtacWorks-main] Checkng input files for compatibility
Building model: resnet ...
Loading model weights from ./models/model.pth.tar...
Finished loading.
INFO:2020-06-10 16:37:06,095:AtacWorks-model_utils] Compiling model in DistributedDataParallel
INFO:2020-06-10 16:37:06,181:AtacWorks-model_utils] Compiling model in DistributedDataParallel
INFO:2020-06-10 16:37:06,231:AtacWorks-model_utils] Compiling model in DistributedDataParallel
INFO:2020-06-10 16:37:06,232:AtacWorks-model_utils] Compiling model in DistributedDataParallel
Finished building.
Inference -------------------- [ 0/29] 
Inference time taken:  189.338s (Load    9.037s,Prediction  179.972s)
INFO:2020-06-10 16:40:22,944:AtacWorks-main] Waiting for writer to finish...
Writing the output to bigwig files
Saving config file to ./infer_config.yaml...


Note: `infer_config.yaml` is set up to use multiple GPUs. If you are using a single GPU, edit `infer_config.yaml` to change the line `gpu: "None"` to read `gpu: 0`.

The inference results will be saved in the folder `output_latest`. This folder will contain four files: 
1. `NK_inferred.track.bedGraph` 
2. `NK_inferred.track.bw` 
3. `NK_inferred.peaks.bedGraph`. 
4. `NK_inferred.peaks.bw`

`NK_inferred.track.bedGraph` and `NK_inferred.track.bw` contain the denoised ATAC-seq track. `NK_inferred.peaks.bedGraph` and `NK_inferred.peaks.bw` contain the positions in the genome that are designated as peaks (the model predicts that the probability of these positions being part of a peak is at least 0.5)

To change any of the parameters for inference with the deep learning model, you can edit the parameters in `configs/infer_config.yaml` or `configs/model_structure.yaml` and run the command above. 

Type `python $atacworks/scripts/main.py infer --help` for an explanation of the parameters.

If you are using your own model instead of the one provided, edit `configs/infer_config.yaml` to supply the path to your model under `weights_path`, in place of `model.pth.tar`.

## Step 8: Format peak calls

Delete peaks that are shorter than 20 bp in leangth, and format peak calls in BED format with coverage statistics and summit calls:

In [21]:
!python $atacworks/scripts/peaksummary.py \
    --peakbw inference_output_latest/NK_inferred.peaks.bw \
    --trackbw inference_output_latest/NK_inferred.track.bw \
    --prefix NK_inferred.peak_calls \
    --out_dir inference_output_latest \
    --minlen 20

INFO:2020-06-10 18:19:13,109:AtacWorks-peaksummary] Writing peaks to bedGraph file inference_output_latest/NK_inferred.peak_calls.bedGraph
INFO:2020-06-10 18:19:13,468:AtacWorks-peaksummary] Reading peaks
INFO:2020-06-10 18:19:13,532:AtacWorks-peaksummary] Calculating peak statistics
INFO:2020-06-10 18:20:29,283:AtacWorks-peaksummary] reduced number of peaks from 225182 to 26575.
INFO:2020-06-10 18:20:29,283:AtacWorks-peaksummary] Writing peaks to BED file inference_output_latest/NK_inferred.peak_calls.bed
INFO:2020-06-10 18:20:29,551:AtacWorks-peaksummary] Deleting bedGraph file


In [2]:
!head tutorial2/inference_output_latest/NK_inferred.peak_calls.bed

#chrom	start	end	len	mean	max	relativesummit	summit
chr1	10060	10363	303	32.660064697265625	54.0	60	10120
chr1	565575	566189	614	81.98696899414062	207.0	321	565896
chr1	569638	570165	527	73.18785858154297	176.0	273	569911
chr1	713600	714786	1186	376.9924011230469	1283.0	531	714131
chr1	762280	763421	1141	143.57669067382812	522.0	594	762874
chr1	805037	805620	583	47.64665603637695	100.0	188	805225
chr1	839713	840498	785	180.5477752685547	583.0	398	840111
chr1	856259	856826	567	38.50440979003906	49.0	194	856453
chr1	877933	878141	208	31.360576629638672	39.0	113	878046


This produces a file `output_latest/NK_inferred.peak_calls.bed` with 8 columns:
1. chromosome
2. start position of peak
3. end position of peak
4. length of peak (bp)
5. Mean coverage over peak
6. Maximum coverage in peak
7. Position of summit (relative to start)
8. Position of summit (absolute)

For more information type `python $atacworks/scripts/peaksummary.py --help`

## Appendix 1: Output the peak probabilities in inference instead of peak calls

The model predicts the probability of every position on the genome being part of a peak. In the above command, we take a cutoff of 0.5, and output the positions of regions where the probability is greater than 0.5. To output the probability for every base in the genome without any cutoff, we use the following command:


In [24]:
!atacworks denoise \
    --input_files NK.50_cells.h5 \
    --sizes_file $atacworks/data/reference/hg19.auto.sizes \
    --config configs/infer_config.yaml \
    --config_mparams configs/model_structure.yaml \
    --infer_threshold None

INFO:2020-06-10 18:23:37,143:AtacWorks-main] Checkng input files for compatibility
Building model: resnet ...
Loading model weights from ./models/model.pth.tar...
INFO:2020-06-10 18:23:46,923:AtacWorks-model_utils] Compiling model in DistributedDataParallel
INFO:2020-06-10 18:23:47,028:AtacWorks-model_utils] Compiling model in DistributedDataParallel
Finished loading.
INFO:2020-06-10 18:23:47,057:AtacWorks-model_utils] Compiling model in DistributedDataParallel
INFO:2020-06-10 18:23:47,155:AtacWorks-model_utils] Compiling model in DistributedDataParallel
Finished building.
Inference -------------------- [ 0/29] 
Inference time taken:  184.018s (Load    6.425s,Prediction  177.351s)
INFO:2020-06-10 18:26:57,449:AtacWorks-main] Waiting for writer to finish...
Writing the output to bigwig files


The inference results will be saved in the folder `inference_output_latest`. This folder will contain the same 4 files described in Step 7. However, `NK_inferred.peaks.bedGraph` and `NK_inferred.peaks.bw` will contain the probability of being part of a peak, for every position in the genome. This command is significantly slower, and the `NK_inferred.peaks.bedGraph` file produced by this command is larger than the file produced in Step 7.

The above command is useful in the following situations:
1. To calculate AUPRC or AUROC metrics.
2. If you are not sure what probability threshold to use for peak calling and want to try multiple thresholds.
3. If you wish to use the MACS2 subcommand `macs2 bdgpeakcall` for peak calling.

To call peaks from the probability track generated by this command, you can use `macs2 callpeak` from MACS2 (link) with the following command:

In [27]:
!macs2 bdgpeakcall -i inference_output_latest/NK_inferred.peaks.bedGraph -o inference_output_latest/inferred.peaks.narrowPeak -c 0.5

INFO  @ Wed, 10 Jun 2020 20:58:06: Read and build bedGraph... 
INFO  @ Wed, 10 Jun 2020 21:00:10: Call peaks from bedGraph... 
INFO  @ Wed, 10 Jun 2020 21:00:21: Write peaks... 
INFO  @ Wed, 10 Jun 2020 21:00:21: Done 


Where `0.5` is the probability threshold to call peaks. Note that the summit calls and peak sizes generated by this procedure will be slightly different from those produced by steps 7-8.